In [ ]:
from desaggregation import *
from value_based_solver import *
from solver_reporter import *
from sklearn import metrics

# Data

In [ ]:
data_dir = '../dane'
mod_name = '' 

In [ ]:
data = pd.read_csv(f'{data_dir}/selected_data{mod_name}.csv').set_index('Year')
# data

In [ ]:
cri_forecast = pd.read_csv(f'{data_dir}/pl_forecast{mod_name}.csv').set_index('Year')
#cri_forecast

In [ ]:
levels_ranges = pd.read_csv(f'{data_dir}/levels_ranges{mod_name}.csv').set_index('idx')
is_gain = levels_ranges.loc['is_gain']
cost_criteria = np.where(is_gain < 1.0)[0]

levels_ranges = 0.01 * levels_ranges # as percents
is_gain

In [ ]:
def split_data(all_data):
    __demand_data = all_data['Electricity demand']
    __criteria_data = all_data.drop('Electricity demand', axis=1)
    return __criteria_data, __demand_data

cri_data, dem_data = split_data(data)
cri_bnd, dem_bnd = split_data(levels_ranges.iloc[:-1])

# cri_data = cri_data.drop(cri_data.columns[-1], axis=1)
# cri_bnd = cri_bnd.drop(cri_bnd.columns[-1], axis=1)
# cri_forecast = cri_forecast.drop(cri_forecast.columns[-1], axis=1)

# Parameters

## some useful functions

In [ ]:
_alpha = 5
_gamma = 0.01

dem_name = 'Zapotrzebowanie el.'
cri_names = [
"PKB",
#"PKB UE",
"Bezrobocie",
"Populacja",
"HDD",
"CDD",
"ODEX",
"Cena węgla kamienego",
"Cena węgla brunatnego",
"Cena energii"
]

In [ ]:
def usual_valbas(criteria_data, demand_data, split_year):
    global _alpha
    global _gamma
    global cost_criteria

    valbas = Desaggregation(criteria_data, demand_data, split_year)
    valbas.alpha  =  _alpha
    valbas.alpha_i = _alpha
    valbas.gamma  =  _gamma
    valbas.gamma_i = _gamma
    valbas.cost_criteria = cost_criteria[:-1]
    valbas.epsilon = 0.005
    
    global cri_bnd, dem_bnd
    valbas.set_levels(cri_bnd, dem_bnd)

    return valbas

In [ ]:
def value_function_formula(solver):
    print('Y = ', end='')
    for i in range(len(solver.b)):
        b = solver.b[i]
        print("{:0.2f}".format(b), end='')
        print(f" X_{i+1} + ", end='')

In [ ]:
def get_prediction_errors(y_test, y_pred):
    errs = {}
    errs['mae'] = np.round(metrics.mean_absolute_error(y_test, y_pred), 3)
    errs['mape'] = np.round(metrics.mean_absolute_percentage_error(y_test, y_pred), 3)
    errs['rmse'] = np.round(metrics.mean_squared_error(y_test, y_pred, squared=False), 3)
#    errs['r2'] = np.round(metrics.r2_score(y_test, y_pred), 3)
    return errs

In [ ]:
def get_marginal_value_functions(xs):
    mar_val_fcs =[]
    for i in range(solver._m.num_cri):
        mar_val_fcs.append([xs[i, k] for k in range(solver._m.alpha_i)])
    return mar_val_fcs

def marginal_X( lvls, mar_val, x ):
    arg = np.where(lvls < x )[0]
    if len(arg)==0 or arg[-1] == len(lvls)-1:
        return mar_val[-1]
    _ar = arg[-1]
    a1 = mar_val[_ar]
    a2 = mar_val[_ar+1]
    a1 = 1 if a1 == 0 else a1
    return a2/a1 * (x - lvls[_ar])

def predict_changes(model, weights):
    n = model.num_cri
    X_pct = model.criteria_test.pct_change().dropna().to_numpy()
    preds = []
    for Xj in X_pct:
        preds.append( np.dot(weights, Xj) )
    return preds

def predict_values(begining, pred_pct):
    y_pred = [begining]
    for i in range(len(pred_pct)):
        y_pred.append(y_pred[-1] * (1 + pred_pct[i]))
    return y_pred[1:]


In [ ]:
# cri_data = cri_data.drop(columns=['CDD', 'HDD'])
# cri_forecast = cri_forecast.drop(columns=['CDD', 'HDD'])

# cri_bnd.drop(columns=['CDD', 'HDD'])
# cri_names = cri_names[:3] + cri_names[5:]

# Experiments
## learning on data from 1999 - 2014; test 2015 - 2020

In [ ]:
valbas = usual_valbas(cri_data , dem_data, split_year=2015)

In [ ]:
solver = ValueBasedSolver(valbas)
solver.solve()

In [ ]:
reporter = ValueBasedSolverReporter(solver, dem_name, cri_names, f'train{mod_name}')
reporter.report()

### Post optimization robustness analysis

In [ ]:
for i in range(valbas.num_cri):
    for K in range(1, valbas.alpha_i):
        solver.solve(postOptimize = ("min", i, K))
        solver.solve(postOptimize = ("max", i, K))

reporter.plot_post_opt()

In [ ]:
value_function_formula(solver)
valbas.cost_criteria

## Prediction on testing set

In [ ]:
y_test = valbas.demand_test.iloc[1:]

weights = solver.b
pred_pct = predict_changes(valbas, weights)
y_pred = predict_values(valbas.demand.iloc[-1], pred_pct)

print(pred_pct)
print(y_pred)

#print(pd.DataFrame(np.array([np.trunc(y_pred), list(y_test)])).set_index(y_test.index))

e = get_prediction_errors(y_test, y_pred)
print(f'MAE & ', e['mae'], '\\\\ ')
print(f'MAPE & ', e['mape'], '\\\\ ')
print(f'RMSE & ', e['rmse'], '\\\\ ')

In [ ]:
reporter.plot_predictions(y_pred)

In [ ]:
b_min, b_mean, b_max = solver.get_postopt_b()
x_min, x_mean, x_max = solver.get_postopt_x()

pct_min = predict_changes(valbas, b_min)
pct_mean = predict_changes(valbas, b_mean)
pct_max = predict_changes(valbas, b_max)

y_pred_min = predict_values(valbas.demand.iloc[-1], pct_min)
y_pred_mean = predict_values(valbas.demand.iloc[-1], pct_mean)
y_pred_max = predict_values(valbas.demand.iloc[-1], pct_max)

min_e = get_prediction_errors(y_test, y_pred_min)
mean_e = get_prediction_errors(y_test, y_pred_mean)
max_e = get_prediction_errors(y_test, y_pred_max)

for err in ['mae', 'mape', 'rmse']:
    print(err.upper(), '&', min_e[err],'&', mean_e[err],'&', max_e[err], '\\\\')

reporter.plot_predictions_and_postopt(y_pred, y_pred_min, y_pred_mean, y_pred_max)

# Prediction

In [ ]:
pred_cri_data = pd.concat([cri_data, cri_forecast]) 
pred_dem_data = dem_data.reindex(pred_cri_data.index)
pred_valbas = usual_valbas(pred_cri_data, pred_dem_data, 2020)

solver = ValueBasedSolver(pred_valbas)
solver.solve()

In [ ]:
pred_cri_data.shape

In [ ]:
reporter = ValueBasedSolverReporter(solver, dem_name, cri_names, f"predict{mod_name}")
reporter.report()


In [ ]:
for i in range(valbas.num_cri):
    for K in range(1, valbas.alpha_i):
        solver.solve(postOptimize = ("min", i, K))
        solver.solve(postOptimize = ("max", i, K))

reporter.plot_post_opt()

In [ ]:
value_function_formula(solver)

## forecasts

In [ ]:
pred_pct = predict_changes(pred_valbas, solver.b)
y_pred = predict_values(dem_data.iloc[-1], pred_pct)

b_min, b_mean, b_max = solver.get_postopt_b()
x_min, x_mean, x_max = solver.get_postopt_x()

pct_min = predict_changes(pred_valbas, b_min)
pct_mean = predict_changes(pred_valbas, b_mean)
pct_max = predict_changes(pred_valbas, b_max)

y_pred_min = predict_values(dem_data.iloc[-1], pct_min)
y_pred_mean = predict_values(dem_data.iloc[-1], pct_mean)
y_pred_max = predict_values(dem_data.iloc[-1], pct_max)

reporter.plot_predictions_and_postopt( y_pred, y_pred_min, y_pred_mean, y_pred_max)